In [2]:
import tweepy
import pandas as pd
import time
import sys
import matplotlib.pyplot as plt
import preprocessing as prep
import nltk

In [11]:
def login(key, secret, rate_limit_control):
    auth = tweepy.AppAuthHandler(key, secret)
    api = tweepy.API(auth, wait_on_rate_limit=rate_limit_control, wait_on_rate_limit_notify=rate_limit_control)
    if not api:
        return "Can't Authenticate"
    else:
        return api

In [8]:
def limit_handler(m):
    print("Rate limit reached. Sleeping for {0} minutes. Oh Jeez!".format(m))
    for i in range(m):
        time.sleep(60)
        print("It's been {0} minutes since darkness started.".format(i + 1))

In [54]:
def collect_tweets(user, page, api):
    print("Collecting Tweets for User: {0} -----------------------------------------------------".format(user))
    tweets = []
    for x in tweepy.Cursor(api.user_timeline, id='TheTweetOfGod', count=200, tweet_mode="extended", include_rts="false", exclude_replies="true").pages(page):
        tweets += x
    text = [[tweet._json["full_text"].replace("\n","").replace("\t",""), tweet.favorite_count, tweet.retweet_count] for tweet in tweets]
    user_df = pd.DataFrame(data=text)
    user_df.columns = ["text", "fav", "ret"]
    user_df['score'] = user_df['fav'] + user_df['ret']
    user_df = user_df.drop(['ret', 'fav'], axis=1)
    user_df.to_csv(user + "_tweets.csv")
    print("DONE! {0} tweets collected from {1}".format(len(user_df), user))
    return user_df

In [58]:
def collect_followers(user, page, api):
    folls = []
    for x in tweepy.Cursor(api.followers, count=200, id='TheTweetOfGod').pages(10):
        folls +=x
    text = [fol.screen_name for fol in folls]
    foll_df = pd.DataFrame(data=text)
    foll_df.columns = ["name"]
    foll_df.to_csv(user + "_followers.csv")
    return foll_df

In [59]:
def collect_favs(user, api):
    try:
        users_df = pd.read_csv(user + "_followers.csv")
    except:
        print(user + "_followers.csv can not be found!")
        sys.exit(1)
    # collect favorited tweets of followers of the user
    print("Total number of users: {0}".format(len(users_df)))
    x = 0
    started = False
    while started == False:
        try:
            favs = api.favorites(users_df.at[x, 'name'], count=200, tweet_mode="extended")
            started = True
        except tweepy.RateLimitError as e:
            limit_handler(15)
            continue
        except tweepy.TweepError as e:
            if str(e) == "Not authorized.":
                print("XXXXX NOT AUTHORIZED XXXXX")
                x = x + 1
    for i in range(x+1,users_df.shape[0]):
        try:
            favs = favs + api.favorites(users_df.at[i, 'name'], count=200, tweet_mode="extended")
            print("Collecting user {0}".format(i))
        except tweepy.RateLimitError as e:
            limit_handler(15)
            continue
        except tweepy.TweepError as e:
            if str(e) == "Not authorized.":
                print("XXXXX NOT AUTHORIZED XXXXX")
            else:
                print(str(e))
        except e:
            print(e)
            #sys.exit(1)
    print("DONE! {0} favorited tweets in the pocket".format(len(favs)))
    text = [[fav.full_text.replace("\n","").replace("\t","")] for fav in favs if fav.lang=="en"]
    fav_df = pd.DataFrame(data=text)
    fav_df.columns = ["text"]
    print("We are left with {0} tweets after language filter.".format(len(fav_df)))
    fav_df.to_csv(user + "_folls_favs_extented3.csv")
    return fav_df

In [7]:
def df_tokenize(df):
  df = prep.remove_url(df)
  df = prep.remove_mentions(df)
  df = prep.remove_num(df)
  df = prep.remove_punc(df)
  df = prep.remove_repeat(df)
  df = prep.get_low(df)
  df = df.apply(nltk.word_tokenize)
  return df

In [8]:
def clense(df):
  df = prep.remove_url(df)
  df = prep.remove_mentions(df)
  df = prep.remove_num(df)
  df = prep.remove_punc(df)
  df = prep.remove_repeat(df)
  df = prep.get_low(df)
  return df

In [12]:
api = login("n5TRBJDT6JZ1MkTNwfaytYQ6w", "E5lGRx2vLESZr7k0wvZd40hR9tLQgJyv1rWJKZLvihhq99c69G", False)

In [175]:
folowers_df = collect_followers("TheTweetOfGod",10, api)
folowers_df

KeyboardInterrupt: 

In [33]:
favs_df = collect_favs("TheTweetOfGod", api)
favs_df

Total number of users: 2000
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT

Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It

XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XX

It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness st

XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT 

It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes 

XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
XX

Rate limit reached. Sleeping for 15 minutes. Oh Jeez!
It's been 1 minutes since darkness started.
It's been 2 minutes since darkness started.
It's been 3 minutes since darkness started.
It's been 4 minutes since darkness started.
It's been 5 minutes since darkness started.
It's been 6 minutes since darkness started.
It's been 7 minutes since darkness started.
It's been 8 minutes since darkness started.
It's been 9 minutes since darkness started.
It's been 10 minutes since darkness started.
It's been 11 minutes since darkness started.
It's been 12 minutes since darkness started.
It's been 13 minutes since darkness started.
It's been 14 minutes since darkness started.
It's been 15 minutes since darkness started.
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
XXXXX NOT AUTHORIZED XXXXX
Rate limit reached. Sleeping for 15 m

In [16]:
user_df = collect_tweets("TheTweetOfGod", 15, api)
user_df

187 tweets from page 1
368 tweets from page 2
544 tweets from page 3
643 tweets from page 4
779 tweets from page 5
910 tweets from page 6
1040 tweets from page 7
1168 tweets from page 8
1288 tweets from page 9
1409 tweets from page 10
1527 tweets from page 11
1644 tweets from page 12
1760 tweets from page 13
1874 tweets from page 14
1975 tweets from page 15
2063 tweets from page 16
Seems there are no mo tweets. We are stopping there. Jeez!
DONE! 2063 tweets collected from TheTweetOfGod


,text,score
0,#HappyDiwali! It must be nice having a religio...,13275
1,Retweet if you disagree.,10169
2,Jesus asked Me to tell Kanye to fuck off.,53559
3,Life is a near-death experience.,29372
4,Fuck this shit.Retweet if you agree.,71519
5,I genuinely am God. You can tell because I'm g...,62987
6,The President of the United States is insane.,97096
7,Other universes are laughing at you.,27128
8,Retweeting this counts as going to church.,27199
9,"Lesbian, gay, bisexual, transgender, queer or ...",56415
